<a href="https://colab.research.google.com/github/drfperez/YoungPhotonicsCongress/blob/main/Freeplanet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Technical Commentary: Photonics and Atmospheric Forward Modeling
1. Modeling Absorption Cross-Sections
The atmospheric_model function acts as a simulator for the spectral transmittance of a complex gaseous medium. In photonics, this is equivalent to defining the transfer function of a biological or chemical optical filter:
 * Molecular Absorption: The Gaussian functions in the code simulate absorption bands. In quantum photonics, these represent the transition probability between molecular energy levels (vibrational-rotational states) triggered by incident photons of specific energies (E = h\nu).
 * Rayleigh Scattering: The 1/\lambda^4 dependency models the scattering of light by particles much smaller than the wavelength. This is the exact same principle that governs signal attenuation in silica optical fibers, where the theoretical loss limit is defined by this intrinsic scattering toward the blue end of the spectrum.
2. Optical Low-Pass Filtering and the "Cloud Deck"
The logic np.maximum(spectrum, cloud_deck) is a non-linear operation that simulates optical opacity.
 * Photonic Analogy: It acts as a signal clipper or an optical floor. In a scattering medium, clouds act as a hard boundary that prevents photons from probing deeper layers. This effectively "mutes" high-amplitude absorption features, similar to how high turbidity in a fluid limits the penetration depth of a laser beam.
3. Precision Infrared Spectroscopy (NIR/SWIR)
The wavelength grid (0.6 to 5.0 μm) spans the Near-Infrared (NIR) and Short-Wave Infrared (SWIR) regions. In photonics, this range is critical because:
 * It offers high transparency compared to the visible spectrum (less Rayleigh noise).
 * It contains the fundamental "vibrational fingerprints" of organic molecules. The JWST NIRSpec instrument functions as a high-precision diffraction grating spectrometer, mapping photons to specific pixels on a detector with nanometric wavelength accuracy.
4. Interactive Curve Fitting and SNR
The use of interactive sliders to match the "Forward Model" to "JWST Data" is a practical exercise in optical parameter estimation.
 * Uncertainty (Error Bars): The error_bars represent the Shot Noise (Poisson noise) and detector read noise. In photonic sensing, adjusting a model to noisy data is essential for determining the Limit of Detection (LOD) for trace gases like water or carbon dioxide in distant environments.
Summary
This script is a prime example of inverse optical engineering: we analyze the filtered light detected by the telescope to reconstruct the physical and chemical properties of the "filter" (the planet's atmosphere).

In [ ]:

import numpy as np
import plotly.graph_objects as go

# -----------------------------
# Wavelength grid & data
# -----------------------------
wavelength = np.array([
    0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,
    2.2,2.4,2.6,2.8,3.0,3.3,3.6,4.0,4.3,4.6,5.0
])

data = np.array([
    2.011,2.012,2.013,2.015,2.018,2.022,2.030,2.045,2.062,2.050,
    2.040,2.035,2.030,2.028,2.027,2.030,2.034,2.038,2.043,2.048,
    2.055,2.062,2.075,2.095,2.070,2.060
])

error = np.full_like(data, 0.006)

# -----------------------------
# Forward model
# -----------------------------
def model(rayleigh_amp, h2o_amp, co2_amp, cloud):
    out = []
    for l in wavelength:
        base = 2.02
        rayleigh = rayleigh_amp / l**4
        h2o = h2o_amp if 1.3 < l < 1.5 else 0
        co2 = co2_amp if 4.2 < l < 4.4 else 0
        out.append(max(base + rayleigh + h2o + co2, cloud))
    return np.array(out)

# -----------------------------
# Slider parameter values
# -----------------------------
rayleigh_vals = np.linspace(0.0, 0.006, 15)
h2o_vals = np.linspace(0.0, 0.06, 15)
co2_vals = np.linspace(0.0, 0.08, 15)
cloud_vals = np.linspace(2.00, 2.04, 15)

# -----------------------------
# Precompute frames
# -----------------------------
frames = []
for r in rayleigh_vals:
    for h in h2o_vals:
        frames.append(go.Frame(
            data=[
                go.Scatter(y=model(r, h, 0.045, 2.015))
            ],
            name=f"r={r:.4f},h={h:.3f}"
        ))

# -----------------------------
# Base figure
# -----------------------------
fig = go.Figure(
    data=[
        go.Scatter(
            x=wavelength, y=data,
            error_y=dict(type="data", array=error, visible=True),
            mode="markers",
            name="JWST Data"
        ),
        go.Scatter(
            x=wavelength,
            y=model(0.002, 0.03, 0.045, 2.015),
            mode="lines",
            name="Forward Model"
        )
    ],
    frames=frames
)

# -----------------------------
# Layout with sliders
# -----------------------------
fig.update_layout(
    template="plotly_dark",
    paper_bgcolor="#0f172a",
    plot_bgcolor="#0f172a",
    height=600,
    title="JWST Transmission Spectroscopy — Interactive Forward Model",
    xaxis_title="Wavelength (μm)",
    yaxis_title="Transit Depth (%)",
    sliders=[{
        "active": 0,
        "pad": {"t": 50},
        "steps": [{
            "label": f"H₂O={h2o_vals[i]:.3f}",
            "method": "animate",
            "args": [[f"r={rayleigh_vals[0]:.4f},h={h2o_vals[i]:.3f}"],
                     {"frame": {"duration": 0, "redraw": True},
                      "mode": "immediate"}]
        } for i in range(len(h2o_vals))]
    }]
)

fig.show()

In [ ]:

import numpy as np
import plotly.graph_objects as go

# ==========================================
# 1. SETUP: SYNTHETIC DATA GENERATION
# ==========================================
# We define a wavelength grid (in microns) typical for JWST instruments (e.g., NIRSpec)
wavelength = np.array([
    0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0,
    2.2, 2.4, 2.6, 2.8, 3.0, 3.3, 3.6, 4.0, 4.3, 4.6, 5.0
])

# Synthetic observations (Transit Depth %)
# Ideally, this comes from a file, but here we hardcode it for the demo.
data_points = np.array([
    2.011, 2.012, 2.013, 2.015, 2.018, 2.022, 2.030, 2.045, 2.062, 2.050,
    2.040, 2.035, 2.030, 2.028, 2.027, 2.030, 2.034, 2.038, 2.043, 2.048,
    2.055, 2.062, 2.075, 2.095, 2.070, 2.060
])

# Constant error bar (instrument uncertainty)
error_bars = np.full_like(data_points, 0.006)

# ==========================================
# 2. THE PHYSICAL MODEL
# ==========================================
def atmospheric_model(wav, h2o_strength, co2_strength, rayleigh_strength, cloud_deck):
    """
    Generates a transmission spectrum based on 4 physical parameters.

    1. Base Radius: The solid surface or deep atmosphere.
    2. Rayleigh Scattering: Scattering of light by small particles (blue slope).
    3. Molecules (H2O, CO2): Modeled here as Gaussian absorption features.
    4. Clouds: An opaque deck that hides features below a certain height.
    """
    base_radius = 2.01

    # 1. Rayleigh Slope (Stronger at short wavelengths: 1/lambda^4)
    rayleigh = rayleigh_strength * (1 / wav**4)

    # 2. Molecular Absorption (Gaussian approximations for teaching)
    # H2O feature centered at 1.4 microns
    h2o_feature = h2o_strength * np.exp(-((wav - 1.4)**2) / (2 * 0.1**2))

    # CO2 feature centered at 4.3 microns
    co2_feature = co2_strength * np.exp(-((wav - 4.3)**2) / (2 * 0.15**2))

    # Sum components
    spectrum = base_radius + rayleigh + h2o_feature + co2_feature

    # 3. Cloud Deck (Math: simply take the maximum of spectrum or cloud_level)
    # Clouds act as a floor; you cannot see below them.
    spectrum = np.maximum(spectrum, cloud_deck)

    return spectrum

# ==========================================
# 3. INTERACTION LOGIC (SLIDERS)
# ==========================================
# We will animate the H2O strength to show how fitting works.
h2o_values = np.linspace(0.0, 0.08, 40) # 40 steps for smooth animation

# Pre-compute the frames for the slider
frames = []
for h_val in h2o_values:
    # We fix other parameters to "reasonable" values to isolate the effect of Water
    y_model = atmospheric_model(
        wavelength,
        h2o_strength=h_val,
        co2_strength=0.045,
        rayleigh_strength=0.002,
        cloud_deck=2.015
    )

    frames.append(go.Frame(
        data=[go.Scatter(x=wavelength, y=y_model)],
        name=f"{h_val:.3f}" # Frame name matches slider label
    ))

# ==========================================
# 4. VISUALIZATION (PLOTLY)
# ==========================================
fig = go.Figure()

# --- Trace 1: The Observed Data (Static) ---
fig.add_trace(go.Scatter(
    x=wavelength, y=data_points,
    error_y=dict(type="data", array=error_bars, visible=True, color="#94a3b8"),
    mode="markers",
    marker=dict(color="#38bdf8", size=8, line=dict(width=1, color="white")),
    name="JWST Observations"
))

# --- Trace 2: The Forward Model (Dynamic) ---
# This acts as the initial state before the user touches the slider
initial_h2o = 0.0
initial_model = atmospheric_model(wavelength, initial_h2o, 0.045, 0.002, 2.015)

fig.add_trace(go.Scatter(
    x=wavelength, y=initial_model,
    mode="lines",
    line=dict(color="#facc15", width=4), # Bright yellow for contrast
    name="Atmosphere Model"
))

# --- Didactic Annotations (Teaching Aids) ---
# Highlight the H2O Band
fig.add_vrect(
    x0=1.2, x1=1.6,
    fillcolor="rgba(56, 189, 248, 0.1)", layer="below", line_width=0,
    annotation_text="Water (H₂O)", annotation_position="top left"
)

# Highlight the CO2 Band
fig.add_vrect(
    x0=4.0, x1=4.6,
    fillcolor="rgba(251, 146, 60, 0.1)", layer="below", line_width=0,
    annotation_text="Carbon Dioxide (CO₂)", annotation_position="top left"
)

# Text annotation for Rayleigh Slope
fig.add_annotation(
    x=0.8, y=2.03,
    text="Rayleigh Scattering<br>(Hazy Sky)",
    showarrow=True, arrowhead=1, ax=0, ay=-40,
    font=dict(color="#cbd5e1", size=10)
)

# --- Layout & Slider Configuration ---
fig.frames = frames

fig.update_layout(
    template="plotly_dark",
    title=dict(
        text="<b>Interactive Exoplanet Atmosphere</b><br>Adjust the Water Abundance to fit the data",
        font=dict(size=20)
    ),
    xaxis=dict(title="Wavelength (microns)", gridcolor="#334155"),
    yaxis=dict(title="Transit Depth (%)", gridcolor="#334155"),
    paper_bgcolor="#0f172a", # Slate-900 (Modern Dark)
    plot_bgcolor="#0f172a",
    height=600,
    hovermode="x unified",

    # Define the Slider
    sliders=[{
        "active": 0,
        "currentvalue": {"prefix": "Water Strength: "},
        "pad": {"t": 50},
        "steps": [{
            "label": f"{h:.3f}",
            "method": "animate",
            "args": [
                [f"{h:.3f}"], # Match the frame name
                {"mode": "immediate", "frame": {"duration": 0, "redraw": True}, "transition": {"duration": 0}}
            ]
        } for h in h2o_values]
    }]
)

fig.show()